In [ ]:
print('Setup complete.')

# Lab 02: Real-World MCPS - Git and Playwright

## Learning Objectives
- Understand how MCPS can wrap complex, real-world tools like Git and Playwright
- Implement a mock agent that uses a `git` MCP server to check the status of a repository
- Implement a mock agent that uses a `playwright` MCP server to interact with a web page
- See how agents can perform complex, stateful operations through the MCPS protocol

## Setup

In [ ]:
from typing import List, Dict, Any, Callable
from dataclasses import dataclass, field
import json

## Part 1: A Mock `git` MCP Server

Imagine an MCP server that provides access to Git commands. An agent could use this to read files, check for changes, and even commit them, all without having direct shell access.

In [ ]:
# --- Mock MCP Server Infrastructure (from Lab 01) ---
@dataclass
class MCPResource:
    name: str
    description: str
    handler: Callable[..., Any]

class MockMCPServer:
    def __init__(self, name: str):
        self.name = name
        self.resources: Dict[str, MCPResource] = {}

    def register_resource(self, resource: MCPResource):
        self.resources[resource.name] = resource

    def list_resources(self) -> List[Dict[str, str]]:
        return [{'name': r.name, 'description': r.description} for r in self.resources.values()]

    def execute_resource(self, name: str, args: Dict) -> Any:
        if name not in self.resources: return f'Error: Resource "{name}" not found.'
        try:
            return self.resources[name].handler(**args)
        except Exception as e: return f'Error: {e}'

# --- Mock Git Tools ---
def git_status(repo_path: str) -> str:
    # In a real system, this would run `git status` in the specified directory
    if repo_path == '/path/to/my/project':
        return 'On branch main\nYour branch is up to date with \'origin/main\'.\n\nChanges not staged for commit:\n  (use "git add <file>..." to update what will be committed)\n  (use "git restore <file>..." to discard changes in working directory)\n\tmodified:   README.md\n'
    return f'Error: Not a git repository: {repo_path}'

# --- Setup the Git MCP Server ---
git_server = MockMCPServer(name='gitmcp')
git_server.register_resource(MCPResource(
    name='git_status',
    description='Shows the working tree status of a Git repository.',
    handler=git_status
))

### Using the `git` Server

In [ ]:
# An agent wants to check for modified files in a project.
query = 'Are there any modified files in the repo at /path/to/my/project?'

# Mock LLM logic to generate a tool call
tool_call = {
    'server': 'gitmcp',
    'resource': 'git_status',
    'args': {'repo_path': '/path/to/my/project'}
}

print(f'--- Agent wants to run: {tool_call["resource"]}("{tool_call["args"]["repo_path"]}") ---\n')

# Execute the call
result = git_server.execute_resource(tool_call['resource'], tool_call['args'])

print(f'Observation:
{result}')

## Part 2: A Mock `playwright` MCP Server

Playwright is a tool for browser automation. An MCP server wrapping Playwright would allow an agent to navigate web pages, fill out forms, and extract information—a powerful capability for web-based tasks.

In [ ]:
# --- Mock Playwright Tools ---
class MockBrowserPage:
    def __init__(self):
        self.url = ''
        self.content = ''

    def navigate(self, url: str) -> str:
        self.url = url
        if 'example.com' in url:
            self.content = '<html><body><h1>Welcome to Example.com</h1><p>This is a test page.</p></body></html>'
            return f'Successfully navigated to {url}'
        return f'Error: Could not navigate to {url}'

    def get_content(self) -> str:
        return self.content

# A single browser instance that the server manages
browser_page = MockBrowserPage()

# --- Setup the Playwright MCP Server ---
playwright_server = MockMCPServer(name='playwright')
playwright_server.register_resource(MCPResource(
    name='navigate',
    description='Navigates the browser to a specific URL.',
    handler=browser_page.navigate
))
playwright_server.register_resource(MCPResource(
    name='get_content',
    description='Returns the full HTML content of the current page.',
    handler=browser_page.get_content
))

### Using the `playwright` Server

In [ ]:
# An agent wants to find the title of a webpage.
goal = 'Find the H1 title on the page http://example.com'

# This would be a ReAct or Plan-and-Execute agent loop.
# Step 1: Navigate to the page
tool_call_1 = {'server': 'playwright', 'resource': 'navigate', 'args': {'url': 'http://example.com'}}
result_1 = playwright_server.execute_resource(tool_call_1['resource'], tool_call_1['args'])
print(f'Step 1 Observation: {result_1}')

# Step 2: Get the content of the page
tool_call_2 = {'server': 'playwright', 'resource': 'get_content', 'args': {}}
result_2 = playwright_server.execute_resource(tool_call_2['resource'], tool_call_2['args'])
print(f'\nStep 2 Observation: Got page content (length: {len(result_2)})')

# Step 3: The agent's LLM would parse the HTML to find the H1 tag.
# (This step is internal to the agent, not a tool call)
import re
match = re.search(r'<h1>(.*)</h1>', result_2)
final_answer = match.group(1) if match else 'H1 not found.'

print(f'\nFinal Answer: {final_answer}')

## Exercises

1. **Add a `git_commit` Resource**: Implement a mock `git_commit(repo_path: str, message: str)` function. Register it with the `git_server`. Then, create a sequence of tool calls for an agent that wants to commit the `README.md` file with the message "Update README".
2. **Add a `playwright_click` Resource**: Implement a mock `click(selector: str)` method on the `MockBrowserPage` class. It should return a success message if the selector is valid (e.g., `'a#link'`) and an error otherwise. Register this new resource with the `playwright_server`.
3. **Create a Central Agent Dispatcher**: Write a function `dispatch_tool_call(tool_call: Dict)` that takes a tool call dictionary (containing `server`, `resource`, and `args`). This function should look up the correct server from a dictionary of servers (`{'gitmcp': git_server, 'playwright': playwright_server}`) and execute the resource on the correct one. This simulates how a real agent would manage connections to multiple MCP servers.

## Summary

You learned:
- How complex, real-world applications like **Git** and **Playwright** can be wrapped by MCP servers to provide tools for AI agents.
- That agents can perform **stateful operations** (like navigating a browser and then getting its content) by making a sequence of tool calls.
- This architecture allows agents to operate on real-world systems (like codebases and websites) in a structured and secure way, without needing direct access to the underlying environment.